In [1]:
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

In [2]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [3]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c39df9f1-d43c-4c89-b4ef-54d33d333177;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 66ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [20]:
loader = IncrementalLoader(spark, '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma', 'delta_news/entertainment')
df = loader.get_new_data()

CDF not available — doing full load


25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921639145 which is greater than the next commit timestamp 1745921639145.
25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639501 which is greater than the next commit timestamp 1745921639239.
25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639502 which is greater than the next commit timestamp 1745921639185.
25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639603 which is greater than the next commit timestamp 1745921639436.
25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639604 which is greater than the next commit timestamp 1745921639330.
25/05/02 19:20:03 WARN DeltaHistoryManager: Found Delta commit 7 with a timestamp 1745921639605 which is greater than the next commit timestamp 1745921639185.
25/05/02 19:20:03 WARN DeltaHistoryManager: Fo

In [37]:
processor = NewsProcessor(spark, df)

In [38]:
processor.ensure_schema()
processor.remove_clear_duplicates()
processor.name_to_id()
processor.remove_hidden_duplicates(['url'], ['publishedAt'])
processor.normalize_text(['Title', 'Description', 'Content'])
processor.expand_source()
processor.order_by('publishedAt', ascending=False)

Removed 40 simple duplicate(s)
Removed 3 hidden duplicate(s)


In [39]:
processor.df

url,author,Content,Description,publishedAt,source,Title,urlToImage
http://www.hollyw...,James Hibberd,bella ramsey s el...,bella ramsey brea...,2025-04-28 04:11:45,hollywood-reporter,the last of us ...,https://www.holly...
http://www.vultur...,Keith Phipps,though most of th...,the jackson commu...,2025-04-28 04:00:03,vulture,the last of us ...,https://pyxis.nym...
http://deadline.c...,Natalie Oganesyan,the rock amp ro...,the rock roll h...,2025-04-28 03:33:00,deadline,outkast cyndi la...,https://deadline....
http://www.vultur...,Sara Holdren,the new musical r...,real women have ...,2025-04-28 03:30:23,vulture,bright and bold i...,https://pyxis.nym...
https://www.eonli...,Corinne Heller,originally appear...,new details about...,2025-04-28 02:25:00,eonline.com,gene hackman s fi...,https://media.zen...
https://www.foxne...,Tracy Wright,katy perry fans w...,social media user...,2025-04-28 01:51:00,fox-news,katy perry ridicu...,https://static.fo...
https://wdwnt.com...,Jamie Rowland,according to the ...,a daily updated r...,2025-04-28 01:44:03,wdwnt.com,new york mom suin...,https://media.wdw...
https://www.buzzf...,Hannah Marder,NULL,wouldn t it be f...,2025-04-28 01:31:02,buzzfeed,it s always fat ...,https://img.buzzf...
https://www.justj...,NULL,rupert grint and ...,congratulations,2025-04-28 01:28:20,just-jared,rupert grint lo...,https://cdn01.jus...
https://www.cbsne...,Kiki Intarasuwan,jiggly caliente ...,bianca castro ara...,2025-04-28 00:19:00,cbs-news,jiggly caliente ...,https://assets1.c...
